In [13]:
import os
import re
import io
import csv
import shutil
import datetime
import time
import sys
import mysql.connector
from mysql.connector import MySQLConnection, Error
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')

except_txt_path = '/Users/Jackie/Desktop/except_txt/'
err_path = '/Users/Jackie/Desktop/error'


In [2]:
#0314 import

import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [3]:
"""
這個function暫時用不到了


'''先進資料庫，確認日期(時間)，將最新的日期(時間)取出，再產生加七天的日期，形成一個
[('rain,date,rain=0,WD'),('rain,date,rain=0,WD'),....]
的陣列再寫進資料庫。'''



def rain_wd_insert(except_txt_path):
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='for1121',
                                       user='root',
                                       password='apple')
        cursor = conn.cursor()
        query = ('''select YMD from rain;''')
        cursor.execute(query)
        ''' 
        use checklist[] to store all the date in the table, later I can use this [] to check data repeat or not,
        but now I'm only use it to get the latest date we have in DB.

        '''
        # 把所有的日期存成list
        checklist=[]
        for ele in cursor:
            for element in ele:
                checklist.append(format(element).decode('utf-8'))
        # 取今天的日期
        td=datetime.datetime.today()
        # 取DB最新的日期資料
        lastest_day = datetime.datetime.strptime(checklist[-1],'%Y-%m-%d')
        #相減得到時間差
        period = td - lastest_day
        # 以天數表示
        p = period.days
        # 若相差天數在7天以上，就執行insert新資料
        # 小於等於七天就不執行，改印出未滿七天，再等幾天吧。
        if p > 7:
            this_week_list=[]
            for i in range(0,7):
                dateymd = (lastest_day + datetime.timedelta(days = i+1 )).date()
                datewd = dateymd.strftime('%a')
                temp=[]
                temp.append('rain')
                temp.append(str(dateymd))
                temp.append('0')
                temp.append(datewd)
                mytuple = tuple(temp)
                this_week_list.append(mytuple)
            for ele in this_week_list:         
                query = ('''insert into {r[0]} (YMD,rain,WD)values ('{r[1]}','{r[2]}','{r[3]}');'''.format(r=ele))
                cursor.execute(query)
                print ('已經成功insert '+ele[1])
            conn.commit()
             
        else:
            print('還差'+str(7-p)+'天才能執行insert table rain喔！')
    except Error as e:
        print(e)
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'insert_rain.txt','w') as here3:
            here3.write(u'insert_rain exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
    finally:
        cursor.close()
        conn.close()
"""

In [4]:
def splitdata(rawdata_path,splited_data_path,except_txt_path):
    try:
        if not os.path.exists(splited_data_path):
            os.makedirs(splited_data_path)
        templist= []
        for ele in open(rawdata_path,'r'):
            templist.append(ele)
        vdname = set()
        for line in templist[2:-2]:
            vdname.add(line.split(',')[0])
        # print list(vdname)

        for ee in vdname:
            with open(splited_data_path+ee+'.csv','w') as wf:
                for line in templist[2:-2]:
                    if ee == line.split(',')[0]:
                        wf.write(line)
    except:
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'split.txt','w') as here3:
            here3.write(u'split exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')


In [5]:
'''VIKPW00 VIKPW00,"光復路(仁愛-信義)-光復南路417-1號前,附掛既設CMS(PJ04)桿",
往北,2016-04-01 00:00:00,2,312.0,397.0,170.0,142.0,0.0,43.22141,1.555,85.005005,'''

#要把"光復路(仁愛-信義)-光復南路417-1號前,附掛既設CMS(PJ04)桿" 中間的','要清掉 



def clean_VIKPW00(clean_data_path):
    csv_content = []
    for line in open(clean_data_path+'VIKPW00.csv','r'):
        liner = line.replace('"光復路(仁愛-信義)-光復南路417-1號前,附掛既設CMS(PJ04)桿"','光復路(仁愛-信義)-光復南路417-1號前附掛既設CMS(PJ04)桿')
    #     print liner
        csv_content.append(liner)
    with open(clean_data_path+'VIKPW00.csv',"w") as wr:
        for oo in csv_content:
            wr.write(oo)
    

In [6]:
'''
1121把今天拿回來的新的VD資料，整個先存進資料庫然後再產csv給昌儒
1128把資料夾的名稱更改成易閱讀的方式，更改成可用於S3。
1208改成要放入輸入及

'''


#檢查data的內容！

def check_data(splited_data_path,err_path,except_txt_path):
    try:
        fileslist=[]
        for root,dirs,files in os.walk(splited_data_path):
            for ele in files:
                if '.csv' in ele:
                    fileslist.append(root+'/'+ele)
        direct=[]
        for f in fileslist:
            # 把csv的資料存進[]
            ff=[]
            with io.open(f,'r',encoding='utf-8') as rfile:
                for line in rfile:
                    ff.append(line)
            count=0
#                 print len(ff[2:-2])
            for ele in ff:
                # direct為已知的方向資料
                direct=['往北','往南','往東','往西','向北','向南','向東','向西','北','南','東','西',\
                        '未指定','正向放置','反向放置','負向放置']
                # 若存在則繼續
                if ele[:-2].split(',')[2] in direct:
                    if ele.split(',')[-4] == '-1.0':
                        count += 1
                        if count == len(ff[2:-2]):
                            import time
                            '''
                            應該要寫個log檔
                            
                            '''
                            print '只有0'
                            print f
                            ct = time.strftime('%Y%m%d',time.localtime())
                            if not os.path.exists(err_path+ct):
                                os.makedirs(err_path+ct)
                            shutil.move(f,err_path+ct)
                # 不存在[]內就印出方向是朝向哪，及檔案名稱。
                else:
                    import time
                    ct = time.strftime('%Y%m%d',time.localtime())
                    if not os.path.exists(err_path+ct):
                        os.makedirs(err_path+ct)
                    '''
                    應該要寫個log檔
                    
                    '''
                    print ele[:-2].split(',')[2]
                    print f 
                    # 
                    shutil.move(f,err_path+ct)
                    break
                colnum = []
                for el in ele[:-2].split(','):
                    if ele[:-2].split(',')[3] == el:
                        YMD = (ele.split(',')[3]).split(' ')[0]
                        time = (ele.split(',')[3]).split(' ')[1]
                        colnum.append(YMD)
                        colnum.append(time)
                    else:
                        colnum.append(el)
                if len(colnum) == 14:
                    pass
                else:
                    import time
                    ct = time.strftime('%Y%m%d',time.localtime())
                    if not os.path.exists(err_path+ct):
                        os.makedirs(err_path+ct)
                        
                    '''
                    應該要寫個log檔
                    
                    '''
                    print('這個row資料異常')
                    print ele
                    print('正常要14欄位，這裡只有'+str(len(colnum))+'欄位 ')
                    shutil.move(f,err_path+ct+'/')
                    break
    except:
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'check_error.txt','w') as here3:
            here3.write(u'check_error exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')

In [7]:
def ETL(in_path,out_path,except_txt_path):
    col1 = u'''EQIPnumber,location,direct,YMD,hour,laneNumber,addTotal,15Total,truckflow,carflow,scooterflow\
        ,avgspeed,avgPercent,avgCarSpace''' + u'\n'
    try:
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        fileslist=[]
        for root,dirs,files in os.walk(in_path):
            for ele in files:
                if '.csv' in ele:
                    fileslist.append(root+'/'+ele)
        for f in fileslist:            
            ff=[]
            Ntemp=[]
            Stemp=[]
            Etemp=[]
            Wtemp=[]
            Utemp=[]
            for line in io.open(f, 'r', encoding='utf-8'):
                ff.append(line)
            for ele in ff:
                if ele.split(',')[-4] == '-1.0':
                    continue
                elif (ele.split(',')[2] =='往北')or(ele.split(',')[2] =='向北')or(ele.split(',')[2] =='北'):
                    sf=[]
                    for el in ele.split(','):
                        if ele.split(',')[3] == el:
                            sf.append(','.join((ele.split(',')[3]).split(' ')))
                        else:
                            sf.append(el)
                    Ntemp.append(','.join(sf)[:-2] + '\n')
                elif (ele.split(',')[2] =='往南')or(ele.split(',')[2] =='向南')or(ele.split(',')[2] =='南'):
                    sf=[]
                    for el in ele.split(','):
                        if ele.split(',')[3] == el:
                            sf.append(','.join((ele.split(',')[3]).split(' ')))
                        else:
                            sf.append(el)
                    Stemp.append(','.join(sf)[:-2] + '\n')
                elif (ele.split(',')[2] =='往東')or(ele.split(',')[2] =='向東')or(ele.split(',')[2] =='東'):
                    sf=[]
                    for el in ele.split(','):
                        if ele.split(',')[3] == el:
                            sf.append(','.join((ele.split(',')[3]).split(' ')))
                        else:
                            sf.append(el)
                    Etemp.append(','.join(sf)[:-2] + '\n')
                elif (ele.split(',')[2] =='往西')or(ele.split(',')[2] =='向西')or(ele.split(',')[2] =='西'):
                    sf=[]
                    for el in ele.split(','):
                        if ele.split(',')[3] == el:
                            sf.append(','.join((ele.split(',')[3]).split(' ')))
                        else:
                            sf.append(el)
                    Wtemp.append(','.join(sf)[:-2] + '\n')
                elif (ele.split(',')[2] =='未指定')or(ele.split(',')[2] =='正向放置')or(ele.split(',')[2] =='反向放置')or(ele.split(',')[2] =='負向放置'):
                    sf=[]
                    for el in ele.split(','):
                        if ele.split(',')[3] == el:
                            sf.append(','.join((ele.split(',')[3]).split(' ')))
                        else:
                            sf.append(el)
                    Utemp.append(','.join(sf)[:-2] + '\n')
            fname = ff[2:-2][0].split(',')[0]   
           
            if Ntemp:
                with io.open(out_path+'N'+fname+'.csv','w',encoding='utf-8') as nfile:
                    nfile.write(col1)
                    for ele in Ntemp:
                        nfile.write(ele)
            if Stemp:
                with io.open(out_path+'S'+fname+'.csv','w',encoding='utf-8') as sfile:
                    sfile.write(col1)
                    for ele in Stemp:
                        sfile.write(ele)
            if  Etemp:
                with io.open(out_path+'E'+fname+'.csv','w',encoding='utf-8') as efile:
                    efile.write(col1)
                    for ele in Etemp:
                        efile.write(ele)
            if Wtemp:
                with io.open(out_path+'W'+fname+'.csv','w',encoding='utf-8') as wfile:
                    wfile.write(col1)
                    for ele in Wtemp:
                        wfile.write(ele)
            if Utemp:
                with io.open(out_path+'U'+fname+'.csv','w',encoding='utf-8') as ufile:
                    ufile.write(col1)
                    for ele in Utemp:
                        ufile.write(ele)
    except:
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'ETL.txt','w') as here3:
            here3.write(u'ETL exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
                            

In [8]:
def check_insert_data(insert_path,err_path,except_txt_path):
    try:
        """
        主要是使用連結資料庫的第三方套件mysql.connector，
        以python去對資料庫做連結，在利用for loop把資料insert到正確的table下。
        1110改正，要在insert前做檢查日期及時間

        """
        import os
        ct=time.strftime('%Y%m%d',time.localtime())

        cnx = mysql.connector.connect(user='root', password='apple', database='for1121')
        cursor = cnx.cursor()
        files = []

        for f in os.listdir(insert_path):
            if os.path.isfile(insert_path + f) & ('.csv' in f):
                files.append(f)

        '''把DB的tablename通通抓出來'''
        tablelist=[]       
        cursor.execute('show tables;')
        for rows in cursor:
            for ele in rows:
                tablelist.append(ele)
        for f in files:
            # 在這裡判斷有沒有存在DB中
            match = re.findall('[A-Z0-9]+', f)
            matchname = str(match[0])
            if matchname in tablelist:
                cursor.execute('select YMD,hour from {};'.format(matchname))
                date_time = []
                for rows in cursor:
                    datelist=[]
                    for ele in rows:
                        datelist.append(str(ele))
                    dati= ','.join(datelist)
                    date_time.append(dati)    
                ff=[]
                with open(insert_path + f,'r') as rfile:
                    for line in rfile:
                        ff.append(line)
                for ele in ff[1:]:
                    if ','.join(ele.split(',')[3:5]) in date_time:
                        print f
                        print(u'資料重複拉！笨蛋'+','.join(ele.split(',')[3:5]))
                        if not os.path.exists(err_path+ct+'datarepeat'):
                            os.makedirs(err_path+ct+'datarepeat')
                        shutil.move(insert_path+f,err_path+ct+'datarepeat/'+f)
                        break
                    else:
                        matchname = str(match[0])
                        # 讀取要寫入的檔案
                        mylist=[matchname,]
                        for el in ele.split(','):
                            mylist.append(str(el).strip())
                        #轉成tuple
                        mytuple = tuple(mylist)

                #這裡要注意的是'{r[1]}','{r[2]}'...等等到r5要加引號是因為在query時就要加''號，因為'00:00:00'沒加''會出錯！！！
                        query = ('''insert into {r[0]} (EQIPnumber,location,direct,YMD,hour,
                        laneNumber,addTotal,15Total,truckflow,carflow,scooterflow,avgspeed,avgPercent
                        ,avgCarSpace)values('{r[1]}','{r[2]}','{r[3]}','{r[4]}','{r[5]}',{r[6]},{r[7]},{r[8]},
                        {r[9]},{r[10]},{r[11]},{r[12]},{r[13]},{r[14]});'''.format(r=mytuple))
                        cursor.execute(query)
                        cnx.commit()
            else:

                print('you have a data not in DB')
                print('it is '+ matchname)
                if not os.path.exists(err_path+ct+'notinDB'):
                    os.makedirs(err_path+ct+'notinDB')
                shutil.move(insert_path+f ,err_path+ct+'notinDB/'+f)


        cursor.close()
        cnx.close()
    except:
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'check_insert_data.txt','w') as here3:
            here3.write(u'check_insert_data exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')

In [9]:
def queryN(result_path,except_txt_path):
    
    """
    1027query改正版，原先的query太多筆，所以現在將要查詢的table整個寫成一個list
    1101query改正，因為給昌儒的資料，方向上太過複雜，所以把方向都統一名稱，但還有'正向放置'，'反向放置'等等要釐清
    1111query改正，因為有些VD位置重複，所以要重新命名位置。
    1115query改正，因應1114去拿回來的資料，把沒有方向性的資料，加上方向，

    """
    '''


    Tablename=['NVMLXB00,向陽路成功橋','UV7230E0,麥帥接塔悠','NVL9SB00,基隆路','NVLKSH00,基隆路','UVTVXH20,成功路四段'\
                 ,'UVTYXQ60,成功路四段']


    for1121的全部data，就從這裡執行，上面的query()要改再改！
    VRGUF00,瑞光路  新增一個瑞光路
    還要記得加入新的Table共31個 
    1122，昌儒改成以後只要這些table，松山跟內湖要分開。
    Tablename=['EVMYN820,民生東路四段','WVMYN820,民生東路四段','EVN8PG20,民生東路四段','WVN8PG20,民生東路四段',\
               'WVNZM960,民權東路四段','EVNZQA20,民權東路四段','EVM8PX60,健康路','WVM8PX60,健康路']
    1128 testing
    1203 更改V6140C0 to V6350G0


    Useless=['EVN8PT60,民生東路二段','WVN8PT60,民生東路二段','EVN5JX20,民生東路二段','WVN5JX20,民生東路二段',\
           'EVMZL960,民生東路三段','WVMZL960,民生東路三段','EVMZLI20,民生東路三段','WVMZLI20,民生東路三段',\
           'EVN9PZ60,民生東路五段','WVN9PZ60,民生東路五段','EVP5KR60,民權東路二段','WVP5KR60,民權東路二段',\
           'EVP6JY60,民權東路二段','WVP6JY60,民權東路二段','EVNUNJ20,民權東路三段','WVNUNJ20,民權東路三段',\
           'EVMZL961,民權東路三段','WVMZL961,民權東路三段','EVP5KR62,民權東路三段','WVP5KR62,民權東路三段',\
           'EVKRM820,八德路二段','WVKRM820,八德路二段','EVKWNV20,八德路三段','WVKWNV20,八德路三段',\
           'EVKZQW20,八德路三段','WVKZQW20,八德路三段','EVL6RM20,八德路三段','WVL6RM20,八德路三段',\
           'NVNUNJ00,敦化北路','SVNUNJ00,敦化北路','SVMCN800,敦化北路','NVLPN800,敦化北路',\
           'NVKWN800,敦化南路一段','SVKWN800,敦化南路一段','SVMZM900,復興北路','NVMQMV00,復興北路',\
           'SVMQMV00,復興北路','SVP6JY40,松江路','NVMEJW40,松江路','SVMEJW40,松江路','NVLRJX00,松江路',\
           'SVLRJX00,松江路','NVN7ST40,塔悠路','SVN7ST40,塔悠路','NVMTR600,三民路','SVMTR600,三民路','NV1050A0,建國北路二段']

    '''
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='for1121',
                                       user='root',
                                       password='apple')
        cursor = conn.cursor()
        # 將要查詢的table以及所屬道路寫成list，方便印出也可以用在query上。
        Tablename=['UVTZY920,成功路四段','WVSHZU60,成功路五段','WVSQYA00,金湖路','NVQ6WK40,成功路2段','SVQ6WK40,成功路2段'\
                   ,'NVQKWL40,成功路2段','SVQKWL40,成功路2段','SVQKWL00,成功路2段'\
                   ,'NVQKWL00,成功路2段','UVSSWC60,成功路3段','NVPBTD70,舊宗路1段','SVQRTE00,舊宗路2段','NVQRTE00,舊宗路2段'\
                   ,'NVRPSV70,舊宗路2段','SVSNRE00,瑞光路','NVSNRE00,瑞光路','NVS6TI00,瑞光路','SVS6TI00,瑞光路'\
                   ,'EVQEUU60,民權東路六段','EVQKWL60,民權東路六段','EVQKWL61,民權東路六段','WVQEUU60,民權東路六段'\
                   ,'WVQKWL60,民權東路六段','WVQKWL61,民權東路六段','NVS9TE00,港墘路','SVS9TE00,港墘路','NVSKTL00,港墘路'\
                   ,'NVRPSV00,堤頂大道1段','SVRPSV00,堤頂大道1段','NVPMSV40,堤頂大道1段','SVPMSV40,堤頂大道1段'\
                   ,'UV6141E0,堤頂大道1段','NVT5QV00,堤頂大道2段','SVT5QV00,堤頂大道2段','SVTXQL00,堤頂大道2段'\
                   ,'NVTXQL00,堤頂大道2段','WVSPRA60,堤頂大道2段','UVS7SN60,堤頂大道2段','WVSGRX60,堤頂大道2段'\
                   ,'EVSGRX60,堤頂大道2段','UVTXQJ20,內湖路1段','UVU5QC60,內湖路1段','UVTHRR20,內湖路1段'\
                   ,'UVT8SZ60,內湖路1段','UVSXTK20,內湖路1段','UV6120C0,麥帥二橋','UV6360G1,麥帥二橋'\
                   ,'UV6130C0,麥帥二橋','UV6360G0,麥帥二橋','UV6140C1,麥帥二橋','UV6350G0,麥帥二橋']
        # 開一個csv檔寫入
        with io.open(result_path,'w',encoding='utf-8')as RDBque:
            # 寫欄位名稱
            RDBque.write("VD編號,道路,位置,道路數,方向,日期,時間,當量,平均速度,平均佔有率,平均車間距,星期"+u'\n')
            i=0
            for ele in Tablename:
                table=[]
                table.append(ele.split(',')[0])
                # 這裡query的寫法參照format的特殊用法，https://pyformat.info/
                select_join=("""select {r[0]}.location,{r[0]}.lanenumber,{r[0]}.direct,{r[0]}.YMD,{r[0]}.hour,
                {r[0]}.15Total  ,{r[0]}.avgspeed ,{r[0]}.avgpercent , {r[0]}.avgCarSpace ,rain.WD from {r[0]} 
                join rain on {r[0]}.YMD=rain.YMD;""".format(r=table))
                cursor.execute(select_join)
                # Using the cursor as iterator
                i+=1
                # 不喜歡用fetchone，fetchmany，fetchall直接取cursor
                for rows in cursor:
                    if (int(rows[6]) <= 10.0):
                        continue 
                    else:
                        RDBque.write(Tablename[i-1][1:]+u',')
                    for element in range(len(rows)):
                        #  如果element是最後一個元素，就寫入換行
                        if element == len(rows)-1:
                            RDBque.write(format(rows[element]).decode('utf-8')+u'\n')
                        elif (rows[element] == "往北")or(rows[element] == "向北")or(rows[element] == "北"):
                            RDBque.write(u'北'+u',')
                        elif (rows[element] == "往南")or(rows[element] == "向南")or(rows[element] == "南"):
                            RDBque.write(u'南'+u',')
                        elif (rows[element] == "往西")or(rows[element] == "向西")or(rows[element] == "西"):
                            RDBque.write(u'西'+u',')
                        elif (rows[element] == "往東")or(rows[element] == "向東")or(rows[element] == "東"):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'VSSWC60,成功路3段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'V6141E0,堤頂大道1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'南'+u',')
                        elif (Tablename[i-1][1:] == 'VS7SN60,堤頂大道2段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'VTXQJ20,內湖路1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'VU5QC60,內湖路1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'VTHRR20,內湖路1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'VT8SZ60,內湖路1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'VSXTK20,內湖路1段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'V6120C0,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'V6360G1,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'V6130C0,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'V6360G0,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'V6140C1,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'V6350G0,麥帥二橋')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'V7230E0,麥帥接塔悠')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'南'+u',')
                        elif (Tablename[i-1][1:] == 'VTVXH20,成功路四段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif (Tablename[i-1][1:] == 'VTYXQ60,成功路四段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'西'+u',')
                        elif (Tablename[i-1][1:] == 'VTZY920,成功路四段')&((rows[element] == "未指定")or(rows[element] == "正向放置")):
                            RDBque.write(u'東'+u',')
                        elif rows[element] == """成功路(文德路-金龍路)-成功路3段152號前路燈桿(已附掛市警局CCTV)""":
                            RDBque.write(u'成功路3段152號前路燈桿'+u',')
                        elif rows[element] == """內湖路(北安路-基湖路)-文湖街對面(捷運柱號0218)路燈桿(台電變電箱旁)""":
                            RDBque.write(u'文湖街對面(捷運柱號0218)路燈桿'+u',') 
                        elif rows[element] == """內湖路(北安路-基湖路)-內湖路1段59號加油站前路燈桿(已附掛市警局CCTV)""":
                            RDBque.write(u'內湖路1段59號加油站前路燈桿'+u',')
                        elif rows[element] == """內湖路(基湖路-港墘路)-西湖捷運站2號出口前路燈桿(TC箱旁)""":
                            RDBque.write(u'西湖捷運站2號出口前路燈桿'+u',')
                        elif rows[element] == """內湖路(基湖路-港墘路)-內湖路1段609號路燈桿(已附掛市警局CCTV)""":
                            RDBque.write(u'內湖路1段609號路燈桿'+u',')
                        elif rows[element] == """內湖路(港墘路-文德路)-港墘捷運站2號出口前號誌桿""":
                            RDBque.write(u'港墘捷運站2號出口前號誌桿'+u',')
                        elif rows[element] == """成功路(康湖路-康寧路三段)-成功路5段康湖路口前CMS桿(4x5字CMS)""":
                            RDBque.write(u'成功路5段康湖路口前CMS桿'+u',')
                        elif rows[element] == """環東大道(南京東至永吉)-正氣橋門型架(往市民大道)""":
                            RDBque.write(u'環東大道正氣橋門型架(往市民大道)'+u',')
                        elif (Tablename[i-1][1:] == 'V6130C0,麥帥二橋')&(rows[element] == "麥二舊宗"):
                            RDBque.write(u'麥二舊宗(橋中段)'+u',')
                        elif (Tablename[i-1][1:] == 'V6350G0,麥帥二橋')&(rows[element] == "堤頂舊宗"):
                            RDBque.write(u'舊宗麥二(內湖端)'+u',')
                        #v6140c0應該是往西才對，位置應該改叫舊宗麥二，但還是要等1121去交控中心詢問。
                        elif (Tablename[i-1][1:] == 'V6140C1,麥帥二橋')&(rows[element] == "麥二舊宗"):
                            RDBque.write(u'麥二舊宗(內湖端)'+u',')
                        elif (Tablename[i-1][1:] == 'V6360G0,麥帥二橋')&(rows[element] == "舊宗麥二"):
                            RDBque.write(u'舊宗麥二(橋中段)'+u',')    
                        elif (Tablename[i-1][1:] == 'V6360G1,麥帥二橋')&(rows[element] == "舊宗麥二"):
                            RDBque.write(u'舊宗麥二(基隆路)'+u',')
                        elif (rows[element] == "未指定")or(rows[element] == "正向放置")or(rows[element] == "反向放置"):
                            print Tablename[i-1][1:]
                        else:
                            RDBque.write(format(rows[element]).decode('utf-8')+u',')

    except Error as e:
        print(e)
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'query_error.txt','w') as here3:
            here3.write(u'query exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')

    finally:
        cursor.close()
        conn.close()


In [10]:
def queryS(result_path,except_txt_path):
    
    """
    1027query改正版，原先的query太多筆，所以現在將要查詢的table整個寫成一個list
    1101query改正，因為給昌儒的資料，方向上太過複雜，所以把方向都統一名稱，但還有'正向放置'，'反向放置'等等要釐清
    1111query改正，因為有些VD位置重複，所以要重新命名位置。
    1115query改正，因應1114去拿回來的資料，把沒有方向性的資料，加上方向，

    """
    '''


    Tablename=['NVMLXB00,向陽路成功橋','UV7230E0,麥帥接塔悠','NVL9SB00,基隆路','NVLKSH00,基隆路','UVTVXH20,成功路四段'\
                 ,'UVTYXQ60,成功路四段']


    for1121的全部data，就從這裡執行，上面的query()要改再改！
    VRGUF00,瑞光路  新增一個瑞光路
    還要記得加入新的Table共31個 
    1122，昌儒改成以後只要這些table，松山跟內湖要分開。
    Tablename=['EVMYN820,民生東路四段','WVMYN820,民生東路四段','EVN8PG20,民生東路四段','WVN8PG20,民生東路四段',\
               'WVNZM960,民權東路四段','EVNZQA20,民權東路四段','EVM8PX60,健康路','WVM8PX60,健康路']
    1128 testing

    Useless=['EVN8PT60,民生東路二段','WVN8PT60,民生東路二段','EVN5JX20,民生東路二段','WVN5JX20,民生東路二段',\
           'EVMZL960,民生東路三段','WVMZL960,民生東路三段','EVMZLI20,民生東路三段','WVMZLI20,民生東路三段',\
           'EVN9PZ60,民生東路五段','WVN9PZ60,民生東路五段','EVP5KR60,民權東路二段','WVP5KR60,民權東路二段',\
           'EVP6JY60,民權東路二段','WVP6JY60,民權東路二段','EVNUNJ20,民權東路三段','WVNUNJ20,民權東路三段',\
           'EVMZL961,民權東路三段','WVMZL961,民權東路三段','EVP5KR62,民權東路三段','WVP5KR62,民權東路三段',\
           'EVKRM820,八德路二段','WVKRM820,八德路二段','EVKWNV20,八德路三段','WVKWNV20,八德路三段',\
           'EVKZQW20,八德路三段','WVKZQW20,八德路三段','EVL6RM20,八德路三段','WVL6RM20,八德路三段',\
           'NVNUNJ00,敦化北路','SVNUNJ00,敦化北路','SVMCN800,敦化北路','NVLPN800,敦化北路',\
           'NVKWN800,敦化南路一段','SVKWN800,敦化南路一段','SVMZM900,復興北路','NVMQMV00,復興北路',\
           'SVMQMV00,復興北路','SVP6JY40,松江路','NVMEJW40,松江路','SVMEJW40,松江路','NVLRJX00,松江路',\
           'SVLRJX00,松江路','NVN7ST40,塔悠路','SVN7ST40,塔悠路','NVMTR600,三民路','SVMTR600,三民路','NV1050A0,建國北路二段']

    '''
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='for1121',
                                       user='root',
                                       password='apple')
        cursor = conn.cursor()
        # 將要查詢的table以及所屬道路寫成list，方便印出也可以用在query上。
        Tablename=['EVMYN820,民生東路四段','WVMYN820,民生東路四段','EVN8PG20,民生東路四段','WVN8PG20,民生東路四段',\
                   'WVNZM960,民權東路四段','EVNZQA20,民權東路四段','EVM8PX60,健康路','WVM8PX60,健康路']
        with io.open(result_path,'w',encoding='utf-8')as RDBque:
            # 寫欄位名稱
            RDBque.write("VD編號,道路,位置,道路數,方向,日期,時間,當量,平均速度,平均佔有率,平均車間距,星期"+u'\n')
            i=0
            for ele in Tablename:
                table=[]
                table.append(ele.split(',')[0])
                # 這裡query的寫法參照format的特殊用法，https://pyformat.info/
                select_join=("""select {r[0]}.location,{r[0]}.lanenumber,{r[0]}.direct,{r[0]}.YMD,{r[0]}.hour,
                {r[0]}.15Total  ,{r[0]}.avgspeed ,{r[0]}.avgpercent , {r[0]}.avgCarSpace ,rain.WD from {r[0]} 
                join rain on {r[0]}.YMD=rain.YMD;""".format(r=table))
                cursor.execute(select_join)
                # Using the cursor as iterator
                i+=1
                # 不喜歡用fetchone，fetchmany，fetchall直接取cursor
                for rows in cursor:
                    if (int(rows[6]) <= 10.0):
                        continue
                    else:
                        RDBque.write(Tablename[i-1][1:]+u',')
                    for element in range(len(rows)):
                        #  如果element是最後一個元素，就寫入換行
                        if element == len(rows)-1:
                            RDBque.write(format(rows[element]).decode('utf-8')+u'\n')
                        elif (rows[element] == "往北")or(rows[element] == "向北")or(rows[element] == "北"):
                            RDBque.write(u'北'+u',')
                        elif (rows[element] == "往南")or(rows[element] == "向南")or(rows[element] == "南"):
                            RDBque.write(u'南'+u',')
                        elif (rows[element] == "往西")or(rows[element] == "向西")or(rows[element] == "西"):
                            RDBque.write(u'西'+u',')
                        elif (rows[element] == "往東")or(rows[element] == "向東")or(rows[element] == "東"):
                            RDBque.write(u'東'+u',') 
                        elif (rows[element] == "未指定")or(rows[element] == "正向放置")or(rows[element] == "反向放置"):
                            print Tablename[i-1][1:]
                        else:
                            RDBque.write(format(rows[element]).decode('utf-8')+u',')

    except Error as e:
        print(e)
        if not os.path.exists(except_txt_path):
            os.makedirs(except_txt_path)
        with open(except_txt_path+'query_error.txt','w') as here3:
            here3.write(u'query exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')

    finally:
        cursor.close()
        conn.close()


In [128]:
in_path = '/Users/Jackie/Desktop/area/Zhongshan'
fileslist=[]
for root,dirs,files in os.walk(in_path):
    for ele in files:
        if '.csv' in ele:
            fileslist.append(root+'/'+ele)
frames = []
for f in fileslist[:-2]:
    print f
    temp = pd.read_csv(f,header=1,skipfooter=2,engine='python')
    frames.append(temp)
    
for f in fileslist[-2:]:
    print f
    temp = pd.read_csv(f,sep=',,',header=1,skipfooter=2,engine='python').dropna(how ="all").reset_index(drop=True)
    frames.append(temp)   


/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0401-0430.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0501-0531.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0601-0630.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0701-0731.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0801-0831.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_0901-0930.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_1001-0031.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_1101-1130.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_16_1201-1231.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_17_0101-0131.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_17_0201-0223.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_17_0224-0302.csv
/Users/Jackie/Desktop/area/Zhongshan/Zhongshan_17_0303-0309.csv


In [129]:
pd.concat(frames,ignore_index=True).to_csv('/Users/Jackie/Desktop/area/Zhongshan/Zhongshan.csv',index=False)

In [135]:
"""

這個function暫時是用不到了
# rain_wd_insert(except_txt_path)"""

# splitdata('/Users/Jackie/Desktop/area/Zhongshan/Zhongshan.csv','/Users/Jackie/Desktop/0310/',except_txt_path)
# clean_VIKPW00('/Users/Jackie/Desktop/0310/') #專為信義寫的ETL
# check_data('/Users/Jackie/Desktop/0310/',err_path,except_txt_path)
# ETL('/Users/Jackie/Desktop/0310/','/Users/Jackie/Desktop/0310/done/',except_txt_path)
# check_insert_data('/Users/Jackie/Desktop/0310/done/',err_path,except_txt_path)
# create_table("/Users/Jackie/Desktop/error20170315notinDB")
# queryN('/Users/Jackie/Desktop/neihu_0224.csv',except_txt_path)
# queryS('/Users/Jackie/Desktop/newVD_0224.csv',except_txt_path)

In [133]:
filelist = []
for roots,dirs,files in os.walk('/Users/Jackie/Desktop/0310/done/'):
    for ele in files:
        if '.csv' in ele:
            filelist.append(ele)
strict_tablename = []
for f in filelist:
    #利用正規表達式，從檔名去找table名
    match = re.findall('[A-Z0-9]+', f)
    tablename = str(match[0])
    strict_tablename.append(tablename)
# check how many
print len(strict_tablename)

# 把各區的VD變成名單
pd.DataFrame(strict_tablename,columns=['ZhongShangVD']).to_csv('ZhongShangVD.csv',index=False)

114


In [52]:
def create_table(dir_path):
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='for1121',
                                       user='root',
                                       password='apple')
        cursor = conn.cursor()
        filelist = []
        
        for roots,dirs,files in os.walk(dir_path):
            for ele in files:
                if '.csv' in ele:
                    filelist.append(ele)
        replace2 = []
        for f in filelist:
            #利用正規表達式，從檔名去找table名
            match = re.findall('[A-Z0-9]+', f)
            tablename = str(match[0])
            replace2.append(tablename)
        for ele in replace2:         
            query = ('''create table {r} (EQIPnumber varchar(255),location varchar(255),direct varchar(25),
            YMD varchar(25),hour varchar(25),laneNumber int(2),addTotal double,15Total double,truckflow double,
            carflow double,scooterflow double,avgspeed double,avgPercent double,avgCarSpace double);'''.format(r=ele))
            cursor.execute(query)
            conn.commit()
    except Error as e:
        print(e)
        with open('/Users/Jackie/Desktop/create_except.txt','w') as here3:
            here3.write(u'create exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
    finally:
        cursor.close()
        conn.close()

In [17]:
create_table(dir_path)